In [1]:
import numpy as np
import pandas as pd
import scipy
import time

import statsmodels.api as sm
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
%matplotlib inline

To do:

Add more features to improve the scores. log regression works better with
Look at PCA

In [2]:
raw_data = pd.read_csv('file:///C:/Users/JChaotogo/Documents/DataSets/KaggleV2-May-2016.csv')

In [3]:
raw_data.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [4]:
raw_data.dtypes
df=raw_data
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [5]:
df['ScheduledDay'] = df['ScheduledDay'].map(lambda x: str(x)[:-10])
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [6]:
df['AppointmentDay'] = df['AppointmentDay'].map(lambda x: str(x)[:-10])
df.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29,2016-04-29,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29,2016-04-29,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29,2016-04-29,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29,2016-04-29,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [7]:
df['ScheduledDay'] = pd.to_numeric(df['ScheduledDay'].str.replace('-', ''))

In [8]:
df['AppointmentDay'] = pd.to_numeric(df['AppointmentDay'].str.replace('-', ''))

In [9]:
df['daystoappt'] = df.AppointmentDay - df.ScheduledDay
df.head(20)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,daystoappt
0,2.987250e+13,5642903,F,20160429,20160429,62,JARDIM DA PENHA,0,1,0,0,0,0,No,0
1,5.589978e+14,5642503,M,20160429,20160429,56,JARDIM DA PENHA,0,0,0,0,0,0,No,0
2,4.262962e+12,5642549,F,20160429,20160429,62,MATA DA PRAIA,0,0,0,0,0,0,No,0
3,8.679512e+11,5642828,F,20160429,20160429,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No,0
4,8.841186e+12,5642494,F,20160429,20160429,56,JARDIM DA PENHA,0,1,1,0,0,0,No,0
5,9.598513e+13,5626772,F,20160427,20160429,76,REPÚBLICA,0,1,0,0,0,0,No,2
6,7.336882e+14,5630279,F,20160427,20160429,23,GOIABEIRAS,0,0,0,0,0,0,Yes,2
7,3.449833e+12,5630575,F,20160427,20160429,39,GOIABEIRAS,0,0,0,0,0,0,Yes,2
8,5.639473e+13,5638447,F,20160429,20160429,21,ANDORINHAS,0,0,0,0,0,0,No,0
9,7.812456e+13,5629123,F,20160427,20160429,19,CONQUISTA,0,0,0,0,0,0,No,2


In [10]:
df['60plus'] = np.where(df.Age>=60,1,0)
df['male'] = np.where(df.Gender=='M',1,0)
df['noshow'] = np.where(df['No-show']=='Yes',1,0)

In [11]:
features = pd.get_dummies(df.Neighbourhood)
df.head(1)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,daystoappt,60plus,male,noshow
0,2.987250e+13,5642903,F,20160429,20160429,62,JARDIM DA PENHA,0,1,0,0,0,0,No,0,1,0,0


In [12]:
features[['age', 'scholarship', 'hypertension', 'diabetes', 'alcoholism', 'handicap', 'sms_received', 'daystoappt', 
          'male', 'noshow','60plus']] = df[['Age', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism', 'Handcap', 
                                   'SMS_received', 'daystoappt', 'male', 'noshow', '60plus']]
features.head(1)

,AEROPORTO,ANDORINHAS,ANTÔNIO HONÓRIO,ARIOVALDO FAVALESSA,BARRO VERMELHO,BELA VISTA,BENTO FERREIRA,BOA VISTA,BONFIM,CARATOÍRA,...,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received,daystoappt,male,noshow,60plus
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [13]:
features[features.noshow==0]

,AEROPORTO,ANDORINHAS,ANTÔNIO HONÓRIO,ARIOVALDO FAVALESSA,BARRO VERMELHO,BELA VISTA,BENTO FERREIRA,BOA VISTA,BONFIM,CARATOÍRA,...,scholarship,hypertension,diabetes,alcoholism,handicap,sms_received,daystoappt,male,noshow,60plus
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,2,0,0,1
8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [14]:
X=features.drop('noshow', 1)
Y=features['noshow']

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = .2)

In [15]:

ridgemodel = LogisticRegression(C=.000001, penalty='l2', class_weight='balanced')
ridgemodel.fit(X_train, y_train)
ridgemodel.score(X_test, y_test)

C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.672984710033475

In [16]:
lassomodel = LogisticRegression(C=.0001, penalty='l1', class_weight='balanced')
lassomodel.fit(X_train, y_train)
lassomodel.score(X_test, y_test)

C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6769655297204379

In [17]:
predy = lassomodel.predict(X_test)
print(pd.crosstab(predy, y_test))
print(classification_report(y_test, predy))

noshow      0     1
row_0              
0       12599  2053
1        5088  2366
              precision    recall  f1-score   support

           0       0.86      0.71      0.78     17687
           1       0.32      0.54      0.40      4419

   micro avg       0.68      0.68      0.68     22106
   macro avg       0.59      0.62      0.59     22106
weighted avg       0.75      0.68      0.70     22106



In [18]:
#Use Gridsearch to find the best parameters for logistic regression.

In [19]:
from sklearn.feature_selection import RFE

selector = RFE(lassomodel)
selector = selector.fit(X, Y)


C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\JChaotogo\AppData\Local

C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\JChaotogo\AppData\Local

In [20]:
selector.ranking_

array([47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31,
       30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14,
       13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1])

In [21]:
rankings = pd.DataFrame({'Features': X.columns, 'Ranking' : selector.ranking_})
rankings2 = rankings.sort_values('Ranking').reset_index()
#Drop some of the features that are not important.
rankings2

,index,Features,Ranking
0,90,60plus,1
1,65,SANTA LÚCIA,1
2,64,SANTA LUÍZA,1
3,63,SANTA HELENA,1
4,62,SANTA CLARA,1
5,61,SANTA CECÍLIA,1
6,60,ROMÃO,1
7,59,RESISTÊNCIA,1
8,58,REPÚBLICA,1
9,57,REDENÇÃO,1


In [22]:
rankings2.drop(columns='index', inplace=True)
rankings2 = rankings2[50:91]
rankings2

,Features,Ranking
50,JESUS DE NAZARETH,7
51,JARDIM DA PENHA,8
52,JARDIM CAMBURI,9
53,JABOUR,10
54,ITARARÉ,11
55,INHANGUETÁ,12
56,ILHAS OCEÂNICAS DE TRINDADE,13
57,ILHA DO PRÍNCIPE,14
58,ILHA DO FRADE,15
59,ILHA DO BOI,16


In [23]:
Xlasso=X

lassofeaturelist = list(rankings2.Features)
for entry in lassofeaturelist:
    Xlasso.drop(columns=[entry], axis=1, inplace=True)
lassofeaturelist

['JESUS DE NAZARETH',
 'JARDIM DA PENHA',
 'JARDIM CAMBURI',
 'JABOUR',
 'ITARARÉ',
 'INHANGUETÁ',
 'ILHAS OCEÂNICAS DE TRINDADE',
 'ILHA DO PRÍNCIPE',
 'ILHA DO FRADE',
 'ILHA DO BOI',
 'ILHA DE SANTA MARIA',
 'ILHA DAS CAIEIRAS',
 'HORTO',
 'GURIGICA',
 'GRANDE VITÓRIA',
 'GOIABEIRAS',
 'FRADINHOS',
 'FORTE SÃO JOÃO',
 'FONTE GRANDE',
 'ESTRELINHA',
 'ENSEADA DO SUÁ',
 'DO QUADRO',
 'DO MOSCOSO',
 'DO CABRAL',
 'DE LOURDES',
 'DA PENHA',
 'CRUZAMENTO',
 'CONSOLAÇÃO',
 'CONQUISTA',
 'COMDUSA',
 'CENTRO',
 'CARATOÍRA',
 'BONFIM',
 'BOA VISTA',
 'BENTO FERREIRA',
 'BELA VISTA',
 'BARRO VERMELHO',
 'ARIOVALDO FAVALESSA',
 'ANTÔNIO HONÓRIO',
 'ANDORINHAS',
 'AEROPORTO']

In [29]:
Xlasso
X_train, X_test, y_train, y_test = train_test_split(Xlasso,Y, test_size = .5)

lassomodel = LogisticRegression(C=.001, penalty='l1', class_weight='balanced')
lassomodel.fit(X_train, y_train)
lassomodel.score(X_test, y_test)

C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6727164157498552

In [26]:
start=time.time()
parameters = {'penalty':['l1'], 'C':[.0000001, .00001,.001,.01,1,100,10000,1000000], 'class_weight':['balanced', None]}
grid = GridSearchCV(lassomodel, parameters, n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(time.time()-start)

C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\model_selection\_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


{'C': 1e-07, 'class_weight': 'balanced', 'penalty': 'l1'}
8.1938955783844


C:\Users\JChaotogo\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [30]:
predy = lassomodel.predict(X_test)
print(pd.crosstab(predy, y_test))
print(classification_report(y_test, predy))

noshow      0     1
row_0              
0       31061  5132
1       12955  6116
              precision    recall  f1-score   support

           0       0.86      0.71      0.77     44016
           1       0.32      0.54      0.40     11248

   micro avg       0.67      0.67      0.67     55264
   macro avg       0.59      0.62      0.59     55264
weighted avg       0.75      0.67      0.70     55264

